In [22]:
my_API_Key='' # Enter Your API Key
my_API_Secret='' # Enter Your API Secret

# For common uses such as receiving price information, there is no need to enter the API keys in full, 
# but it is enough for us as it was placed above: my_API_Key='' and my_API_Secret=''

In [23]:
############################################################
# My API and Secret Key are typed above to save my privacy!

In [24]:
# Importing 'binance' to access the price data
from binance import Client
# Importing pandas to manage the data
import pandas as pd
# Importing datetime to manage the date and time of the prices
import datetime
# Importing pyplot to plot the prices 
from matplotlib import pyplot as plt
# Importing ta library
import ta
import pandas_ta as pta

In [25]:
# Creating a function to retrieve the price data from binance
def give(shareName,timeframe,limit):
    symbol=shareName+'USDT' #for example, 'BTCUSDT'
    API_Key=my_API_Key #Typed above
    API_Secret=my_API_Secret #Typed above
    client = Client(API_Key, API_Secret)
    klines = client.get_historical_klines(symbol=symbol, interval=timeframe, limit=limit)
    return klines

In [26]:
# Creating a function to handle the retrieved data as a dataframe
def create_excel(shareName,timeframe,limit):  
    data=give(shareName,timeframe,limit)
    df=pd.DataFrame(data,columns=['Open Time','Open','High','Low','Close','Volume',
                                    'Close Time','Quote asset volume','Number of trades','Taker buy base asset volume',
                                    'Taker buy quote asset volume','Ignore'])
    df['Datetime'] = pd.to_datetime(df['Open Time'], unit='ms')
    df.index=df['Datetime']
    columns_to_convert = ['Open','High','Low','Close','Volume']
    df[columns_to_convert] = df[columns_to_convert].astype(float)
    excelName=shareName+timeframe+'.xlsx' # for example: 'BTC1h.xlsx'
    df.to_excel(excelName)
    return df

In [27]:
def IchiPlot(shareName,timeframe,limit):
    # Import necessary libraries
    import plotly.graph_objects as go
    import numpy as np

    # 'create_excel' is a function that fetches historical price data for a given stock
    # 'prices' DataFrame contains historical price data for the stock
    prices = create_excel(shareName, timeframe, limit)

    # Infer frequency of data (hourly, daily, etc.)
    freq = pd.infer_freq(prices.index)

    # Generate future date indices for forecast
    future_indices = pd.date_range(start=prices.index[-1], periods=26, freq=freq)

    # Create an empty DataFrame for future data with future date indices
    future_data = pd.DataFrame(index=future_indices)

    # Combine historical and future data
    extended_prices = pd.concat([prices, future_data])

    # Calculate Tenkan-sen (Conversion Line)
    period9_high = prices['High'].rolling(9).max()
    period9_low = prices['Low'].rolling(9).min()
    extended_prices['tenkan_sen'] = (period9_high + period9_low) / 2

    # Calculate Kijun-sen (Base Line)
    period26_high = prices['High'].rolling(26).max()
    period26_low = prices['Low'].rolling(26).min()
    extended_prices['kijun_sen'] = (period26_high + period26_low) / 2

    # Calculate Senkou Span A
    extended_prices['senkou_span_a'] = ((extended_prices['tenkan_sen'] + extended_prices['kijun_sen']) / 2).shift(26)

    # Calculate Senkou Span B
    period52_high = prices['High'].rolling(52).max()
    period52_low = prices['Low'].rolling(52).min()
    extended_prices['senkou_span_b'] = ((period52_high + period52_low) / 2)
    extended_prices['senkou_span_b'] = extended_prices['senkou_span_b'].shift(26)

    # Calculate Chikou Span
    extended_prices['chikou_span'] = extended_prices['Close'].shift(-22)

    # Create a copy of the DataFrame for plotting purposes
    df2 = extended_prices.copy()

    # Label data points based on Senkou Span A and Senkou Span B relationship
    # If Senkou Span A > Senkou Span B, label as 1; otherwise, label as 0
    df1=df2#[52+25:]
    df=df1
    df['label'] = np.where(df['senkou_span_a'] > df['senkou_span_b'], 1, 0)

    # Group data points based on label changes
    df['group'] = df['label'].ne(df['label'].shift()).cumsum()
    df = df.groupby('group')
    dfs = []

    # Split DataFrame into smaller DataFrames based on groups
    for name, data in df:
        dfs.append(data)

    # Define function to fill color based on label
    def fillcol(label):
        if label >= 1:
            return 'rgba(0,250,0,0.4)'  # Green color
        else:
            return 'rgba(250,0,0,0.4)'  # Red color

    # Create a Plotly figure
    fig = go.Figure()

    # Plot each group with filled area between Senkou Span A and Senkou Span B
    for df in dfs:
        fig.add_trace(go.Scatter(x=df.index, y=df['senkou_span_a'],
                                 line=dict(color='rgba(0,0,0,0)')))  # Invisible line for Senkou Span A

        fig.add_trace(go.Scatter(x=df.index, y=df['senkou_span_b'],
                                 line=dict(color='rgba(0,0,0,0)'),
                                 fill='tonexty',  # Fill between Senkou Span A and B
                                 fillcolor=fillcol(df['label'].iloc[0])))  # Fill color based on label of first data point

    '''
    # Plot Senkou Span A and Senkou Span B without filling
    fig.add_trace(go.Scatter(x=df1.index, y=df1['senkou_span_a'],
                             line=dict(color='blue', width=1)))  # Senkou Span A line

    fig.add_trace(go.Scatter(x=df1.index, y=df1['senkou_span_b'],
                             line=dict(color='red', width=1)))  # Senkou Span B line
    
    fig.add_trace(go.Scatter(x=df1.index, y=df1['chikou_span'],
                             line=dict(color='black', width=1)))  # chikou_span line
    
    fig.add_trace(go.Scatter(x=df1.index, y=df1['tenkan_sen'],
                             line=dict(color='yellow', width=1),name='tenkan_sen'))  # tenkan_sen line
    fig.add_trace(go.Scatter(x=df1.index, y=df1['kijun_sen'],
                             line=dict(color='purple', width=1),name='kijun_sen'))  # tenkan_sen line
    ''' 

    # Plot candlestick chart
    fig.add_trace(go.Candlestick(x=df1.index,
                                 open=df1['Open'],
                                 high=df1['High'],
                                 low=df1['Low'],
                                 close=df1['Close']))

    # Update layout of the figure
    title='Ichimoku of '+shareName+' in Timeframe of: '+freq
    fig.update_layout(title=title,showlegend=False, xaxis_rangeslider_visible=False)

    # Show the figure
    fig.show()

In [32]:
IchiPlot('BTC','1h',7*24)
# 7*24 in the 1hour timeframe represents 7*24hour or 7 days (equal to 1 week).  